In [4]:
import requests

import pandas as pd

import openpyxl

print(pd.__version__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
# pd.set_option('future.no_silent_downcasting', True)

2.3.3


In [ ]:
from datetime import datetime, timedelta

def generate_month_year_list(start_year, start_month, n_months=36):
    start_month = datetime(year=start_year, month=start_month, day=1)
    month_year_list = []

    for _ in range(n_months):
        # Format the month and year as "Month Year"
        month_year = start_month.strftime("%B %Y")
        month_year_list.append(month_year)openpyxl
        
        # Move to the previous month
        # Use timedelta to avoid issues with months of varying lengths
        first_day_of_current_month = start_month.replace(day=1)
        start_month = first_day_of_current_month - timedelta(days=1)
    
    return month_year_list

# generate_month_year_list(2024, 8)

In [7]:
# URL of the file to be downloaded
local_filename = 'jpm-emerging-europe-middle-east-afria-disclosure.xlsx'
url = f'https://am.jpmorgan.com/content/dam/jpm-am-aem/emea/gb/en/supplemental/full-portfolio-listing/{local_filename}'

# Send a HTTP GET request to the URL
with requests.get(url, stream=True) as response:
    response.raise_for_status()  # Check for HTTP errors

    # Open a local file with write-binary mode
    with open(local_filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

print(f'File downloaded: {local_filename}')

File downloaded: jpm-emerging-europe-middle-east-afria-disclosure.xlsx


In [8]:
ss = openpyxl.load_workbook(local_filename)
ss['Oct 2025'].title = 'October 2025'
ss['Sep 2025'].title = 'September 2025'
ss['Aug 2025'].title = 'August 2025'
ss['Mar 2025'].title = 'March 2025'
ss['Feb 2025'].title = 'February 2025'
ss['Jan 2025'].title = 'January 2025'
ss['Dec 2024'].title = 'December 2024'
ss['Nov 2024'].title = 'November 2024'
ss['Oct 2024'].title = 'October 2024'
ss['Sep 2024'].title = 'September 2024'
ss['Aug 2024'].title = 'August 2024'
ss.save(local_filename)

In [10]:
description_col = 'Security Description'
security_col = 'Security No.'
symbol_col = 'Symbol'

month_data = pd.read_excel(local_filename, sheet_name='October 2025', skiprows=9, usecols=range(5))
month_data.drop(["% of Fund"], axis=1, inplace=True)
month_data.dropna(inplace=True)
month_data = month_data.sort_values(by=description_col).reset_index(drop=True)
month_data.to_csv("jema_holdings_oct_2025.csv")
display(month_data)

,Holding,Security Description,Market Value,Security No.
0,44158,ABSA GROUP LTD,375877.04,BFX05H3
1,117334,ABU DHABI ISLAMIC BANK PJSC COMMON STOCK AED 1,515464.70,6001728
2,108419,ADES HOLDING CO COMMON STOCK SAR 1,371655.55,BR56KM3
3,420883,ADNOC LOGISTICS & SERVICES COMMON STOCK AED 1.983,500624.93,BRBN103
4,54131,AL RAJHI BANK COMMON STOCK SAR 10,1161252.36,B12LZH9
5,284413,ALDAR PROPERTIES PJSC COMMON STOCK AED 1,530433.75,B0LX3Y2
6,7079,ALDREES PETROLEUM AND TRANSPORT SERVICES CO CO...,220251.89,B128FF8
7,8779,ALKHORAYEF WATER & POWER TECHNOLOGIES CO COMMO...,267799.40,BN33QN1
8,108243,ALPHA BANK SA COMMON STOCK EUR 0.3,323306.19,BTJZ707
9,3753,ANGLOGOLD ASHANTI PLC ZAR,194355.61,BRXH266


In [11]:
jema_symbols_exchanges = pd.read_csv('./jema_symbols_exchanges.csv').dropna(subset=[symbol_col])
jema_symbols_exchanges.reset_index(inplace=True, drop=True)
jema_symbols_exchanges = jema_symbols_exchanges.drop(columns=[description_col])
jema_symbols_exchanges = jema_symbols_exchanges.dropna()
display(jema_symbols_exchanges[jema_symbols_exchanges[security_col] == "0H5837S"])
# display(jema_symbols_exchanges)

,Issuer,Security No.,Symbol,Exchange,Currency,Conv Rate
194,VTB BANK PJSC,0H5837S,VTBR,RUS,RUB,1


In [ ]:
current_month_idx = 10
year = 2025

month_years = generate_month_year_list(year, current_month_idx)
# month_years = ['March 2022', 'February 2022']
# month_years = ['February 2023', 'January 2023'] #, 'December 2022']
# print(month_years)

# set up structure of df
security_col = 'Security No.'
holding_col = 'Holding'
value_col = 'Market Value'
percent_col = '% of Fund'
sheet_name = month_years[0]

jema_data = pd.read_excel(local_filename, sheet_name=sheet_name, skiprows=9, usecols=range(5))
jema_data = jema_data.dropna()
jema_data = jema_data.drop(columns=[holding_col, value_col, percent_col])

# get all unique entries by security_col
for month_year in month_years:
    sheet_name = month_year
    try:
        month_data = pd.read_excel(local_filename, sheet_name=sheet_name, skiprows=9, usecols=range(5))
        month_data = month_data.drop(columns=[holding_col, value_col, percent_col]).dropna()
        month_data[security_col] = month_data[security_col].replace('0H6400S', '2H6442S')

        # fix LUKOIL issue - drop the entries with the wrong security code, they'll be amended below
        month_data = month_data[month_data[security_col] != '0H6400S']
        
        # fix Rosneft issue - drop the entries with the wrong security code, they'll be amended below
        month_data = month_data[month_data[security_col] != '0H6367S']

        # drop subtotal row
        month_data = month_data[month_data[description_col] != 'Subtotal']

        jema_data = pd.concat([jema_data, month_data], ignore_index=True)
    except Exception as e:
        print(f"{sheet_name} failed")

print()

jema_data = pd.merge(jema_data, jema_symbols_exchanges, on=security_col, how='outer')

jema_data = jema_data.drop_duplicates([security_col])
jema_data = jema_data.reset_index(drop=True)
jema_data['Conv Rate'] = jema_data['Conv Rate'].fillna(1)
jema_data = jema_data.fillna('')
jema_data = jema_data.sort_values(by=[description_col], ignore_index=True)
# display(jema_data)

# num_rows = len(jema_data)

# jema_data_old = jema_data

# def read_jema_sheet(jema_data, sheet_name, security_col):
#         month_data = pd.read_excel(local_filename, sheet_name=sheet_name, skiprows=9, usecols=range(5))
#         month_data.drop(["% of Fund"], axis=1, inplace=True)
#         month_data.dropna()
#         month_data = month_data.dropna()
#         month_holding_col = f'{sheet_name}'
#         month_value_col = f'{value_col} {sheet_name}'
#         month_percent_col = f'{percent_col} {sheet_name}'

#         month_data.rename(columns={holding_col: month_holding_col}, inplace=True)
#         month_data.rename(columns={value_col:   month_value_col}, inplace=True)
#         month_data.rename(columns={percent_col: month_percent_col}, inplace=True)
#         month_data = month_data.dropna()

#         # fix LUKOIL issue - amend wrong security code
#         month_data[security_col] = month_data[security_col].replace('0H6400S', '2H6442S')

#         # fix Rosneft issue  - amend wrong security code
#         month_data[security_col] = month_data[security_col].replace('0H6367S', '2H7674S')

#         # fix Novatek issue - amend wrong security code
#         month_data.loc[(month_data[security_col] == '2H6464S') & (month_data[month_holding_col] ==  103572), security_col] = '0H5828S'

#         # drop subtotal row
#         month_data = month_data[month_data[description_col] != 'Subtotal']

#         jema_data = jema_data.merge(month_data[[month_holding_col, security_col]], on=security_col, how='left')
#         jema_data.fillna(0, inplace=True)

#         return jema_data


# for month_year in month_years:
#     sheet_name = month_year
#     try:
#         jema_data = read_jema_sheet(jema_data, sheet_name, security_col)        
#         if(len(jema_data_old) != len(jema_data)):
#             duplicates = jema_data[jema_data[security_col].duplicated(keep=False)]
# descripti
#             print(f"\nRows with duplicate names {sheet_name}:")
#             print(duplicates)

#         jema_data_old = jema_data

#     except Exception as e:
#         print(f"{sheet_name} failed")

# jema_data.to_csv("jema_holdings_full.csv")

jema_data[['Issuer', 'Type']] = (
    jema_data['Security Description']
    .str.split('COMMON STOCK', n=1, expand=True)
)

jema_data['Type'] = jema_data['Type'].mask(jema_data['Type'].isna(), None) \
                                     .apply(lambda x: None if x is None else 'COMMON STOCK' + x)

jema_data = jema_data[[ 'Security Description', 'Issuer', 'Type', 'Security No.']]

display(jema_data)
jema_data.to_csv("jema_symbols_exchanges_new.csv")

,Security Description,Issuer,Type,Security No.
0,ABSA GROUP LTD,ABSA GROUP LTD,None,BFX05H3
1,ABU DHABI COMMERCIAL BANK PJSC COMMON STOCK AED 1,ABU DHABI COMMERCIAL BANK PJSC,COMMON STOCK AED 1,6545464
2,ABU DHABI ISLAMIC BANK PJSC COMMON STOCK AED 1,ABU DHABI ISLAMIC BANK PJSC,COMMON STOCK AED 1,6001728
3,ABU DHABI NATIONAL OIL CO FOR DISTRIBUTION PJS...,ABU DHABI NATIONAL OIL CO FOR DISTRIBUTION PJSC,COMMON STOCK AED 0.08,BYVGM64
4,ADES HOLDING CO COMMON STOCK SAR 1,ADES HOLDING CO,COMMON STOCK SAR 1,BR56KM3
5,ADNOC DRILLING CO PJSC COMMON STOCK,ADNOC DRILLING CO PJSC,COMMON STOCK,BN12D39
6,ADNOC GAS PLC,ADNOC GAS PLC,None,BPJLW35
7,ADNOC LOGISTICS & SERVICES COMMON STOCK AED,ADNOC LOGISTICS & SERVICES,COMMON STOCK AED,6H9334S
8,ADNOC LOGISTICS & SERVICES COMMON STOCK AED 1.983,ADNOC LOGISTICS & SERVICES,COMMON STOCK AED 1.983,BRBN103
9,AKBANK TAS COMMON STOCK TRY 1,AKBANK TAS,COMMON STOCK TRY 1,B03MN70


In [13]:
latest_data = jema_data.iloc[:, :7]
latest_data = latest_data.rename(columns={'Oct 2025': 'Holding'})
latest_data = latest_data[latest_data['Holding'] != 0]
display(latest_data)
latest_data.to_csv("jema.csv")

KeyError: 'Holding'

In [40]:
jema_data[jema_data[security_col] == '0H6400S'] # deleted

,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,October 2024,September 2024,August 2024,July 2024,June 2024,May 2024,April 2024,March 2024,February 2024,January 2024,December 2023,November 2023,October 2023,September 2023,August 2023,July 2023,June 2023,May 2023,April 2023,March 2023,February 2023,January 2023,December 2022,November 2022,October 2022,September 2022,August 2022,July 2022,June 2022,May 2022,April 2022,March 2022


In [41]:
jan23 = pd.read_excel(local_filename, sheet_name='January 2023',  skiprows=9, usecols=range(5)).dropna()
dec22 = pd.read_excel(local_filename, sheet_name='December 2022', skiprows=9, usecols=range(5)).dropna()
mar22 = pd.read_excel(local_filename, sheet_name='March 2022',    skiprows=9, usecols=range(5)).dropna()
apr22 = pd.read_excel(local_filename, sheet_name='April 2022',    skiprows=9, usecols=range(5)).dropna()
jun24 = pd.read_excel(local_filename, sheet_name='June 2024',     skiprows=9, usecols=range(5)).dropna()
apr22 = pd.read_excel(local_filename, sheet_name='April 2022',    skiprows=9, usecols=range(5)).dropna()
# joined = pd.merge(jan23, dec22, on=security_col, how='left')
# joined = mar22.merge(apr22, on=security_col, how='left')
# joined = jun24.merge(apr22, on=security_col, how='left')

sec = '2H6442S'

display(jan23[jan23[security_col] == sec])
display(dec22[dec22[security_col] == sec])
joined = pd.merge(jan23, dec22, on=security_col, how='left')
display(joined)

,Holding,Security Description,Market Value,% of Fund,Security No.


,Holding,Security Description,Market Value,% of Fund,Security No.


,Holding_x,Security Description_x,Market Value_x,% of Fund_x,Holding_y,Security Description_y,Market Value_y,% of Fund_y,Security No.
